### Homework 3 | Generative Adversarial Networks| 15 Points Total | Due October 31 @ 11:59 pm

In this homework you will be taking a look the architecture, loss functions, data generation, and training of different GAN models. First you will build a generic GAN to generate new unconditional samples from MNIST. Then you will build [Pix2Pix](https://phillipi.github.io/pix2pix/) model that we looked at in class previously using an existing dataset. Finally, you will need to generate or collect your own dataset and train a pix2pix model that does conditional image generation.

In [3]:
#Let's start by importing the necessary libraries for this assignment
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output
import torchinfo
import urllib.request
import os
from PIL import Image

In [ ]:
#Let's also check if a GPU is available and set the device accordingly for mac/windows users
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

#### Part 1 | Unconditional GAN

In this section you will define a simple GAN model for unconditional image generation. After training, our expectation is that we should be able to feed in random noise and get back a sample that is convincingly from the distribution of the training dataset.

A GAN is made up of two competing models. These models are generally referred to as the **Generator** and the **Discriminator**. The specific model setup is not defined in [Goodfellow et al.](https://arxiv.org/abs/1406.2661), but we can infer many of the details from the paper and build two MLP style models with a reasonably large capacity. *Reasonably large* is a pretty arbitrary specification. Feel free to experiment with the input/output sizes of the layers in the generator and discriminator to see how it affects the results of your model. 

 Let's start by defining the `Generator` with it's `init` and `forward` methods. This model should just take in a noise vector of `z_dim` and upsample this until it outputs an image the same shape as the MNIST images (28x28).

In [3]:
class Generator(nn.Module):
    def __init__(self, z_dim):
        super(Generator, self).__init__()
        #Store the dimension of the noise vector
        self.z_dim = z_dim

        #Define the layers of the generator
        self.model = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 28 * 28),  # MNIST images are 28x28
            nn.Tanh()  # Output values between -1 and 1
        )

    def forward(self, x):
        return self.model(x).view(-1, 1, 28, 28) # Reshape output to image shape (batch_size, channels, x_dim, y_dim)

Now we will define the `Discriminator` which should take in either a real image from the training set or a synthetic image from the `Generator` and make a binary prediction of whether it is a real or fake image. 

Goodfellow et. al. suggests the use of dropout layers to promote training stability. They also recommend the use of [maxout activations](https://paperswithcode.com/method/maxout), however we will use [Leaky ReLU](https://paperswithcode.com/method/leaky-relu) instead. 

[Leaky ReLU](https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html) looks like the standard ReLU, but has a slight slope for values below 0 to prevent [*vanishing gradients*](https://en.wikipedia.org/wiki/Vanishing_gradient_problem).





In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 512),
            nn.LeakyReLU(0.2), # Leaky ReLU activation function with a slope of 0.2
            nn.Dropout(0.3),  # Dropout layer
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2), # Leaky ReLU activation function with a slope of 0.2
            nn.Dropout(0.3),  # Dropout layer
            nn.Linear(256, 1),  # Output a single value
            nn.Sigmoid()  # Output prediction values between 0 and 1
        )

    def forward(self, img):
        return self.model(img)

Great! Now we have our two models defined. Next we will define the loss function and optimizer for our GAN. Recall that the process for training the GAN model is as follows:

1. Initialize the generator and discriminator models
2. Set the number of training epochs
3. Define the loss function (e.g., Binary Cross-Entropy)
4. Choose optimizers for both the generator and discriminator (e.g., Adam)

5. For each epoch:

    For each batch of real images from the training dataset:
    
        i. Generate random noise (z) as input for the generator
        
        ii. Use the generator to create fake images (G(z))
        
        iii. Compute the discriminator's predictions:
            - Real images: D(real_images)
            - Fake images: D(G(z))
        
        iv. Calculate the loss for the discriminator:
            - Loss_D = - (mean(log(D(real_images))) + mean(log(1 - D(G(z)))))
        
        v. Update the discriminator's weights using the calculated loss
        
        vi. Generate new random noise (z) for the generator
        vii. Compute the discriminator's predictions for the fake images again: D(G(z))
        
        viii. Calculate the loss for the generator:
            - Loss_G = - mean(log(D(G(z))))
        
        ix. Update the generator's weights using the calculated loss

6. End training after the specified number of epochs

First you should initialize your generator and discriminator and set up your optimizers for the generator and discriminator. (1 point)

In [6]:
z_dim = 100

generator = ###Your code here
discriminator = ###Your code here

#We will set the these variables here for use later
n_epochs = 100
batch_size = 64
lr = 0.0002

#The loss function for your model can be defined as a Binary Cross Entropy loss
#For details on why we can use this loss everywhere see: https://towardsdatascience.com/decoding-the-basic-math-in-gan-simplified-version-6fb6b079793
loss_fn = nn.BCELoss()

optimizer_G = optim.Adam(generator.parameters(), lr=lr)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

Before we go much further it it would be nice if we have a function that we can call to plot some random samples with our generator. This will let us check the progress of the optimization as it progresses. 

This function will be called `generate_images(generator, num_images, noise_dim, device)` and it should not have a return value, it should just plot a set of images. (2 points)

In [ ]:
def generate_images(generator, num_images, noise_dim, device):
    #First you need to create a set of random z vectors that have the shape (num_images, noise_dim)
    #Hint: use torch.randn()
    ###Your code here

    #Next we will stop tracking gradients for a bit to generate the images
    #Hint: look up .cpu()
    with torch.no_grad():
        generated_images = ###Your code here

    #Now that you have your generated images you can plot them in a subplot
    #Hint: use plt.subplots()
    """
    Your Code here
    """

Now we have everything we need to define our training loop. Let's also set this up as a function we can call later on. You will need to fill out a number of sections in this training loop function. (5 points)

In [8]:
def train_gan(generator, discriminator, n_epochs, batch_size, lr, loss_fn, optimizer_G, optimizer_D, dataloader, device):
    z_dim = generator.z_dim

    for epoch in tqdm(range(n_epochs)):
        for i, (real_images, _) in enumerate(dataloader):
            # Move real images to device
            real_images = ###Your code here

            #Create labels for real and fake images
            real_labels = torch.ones(real_images.shape[0]).unsqueeze(-1).to(device)
            fake_labels = torch.zeros(real_images.shape[0]).unsqueeze(-1).to(device)

            #Zero out the gradients for the discriminator optimizer
            ###Your code here

            #Have the discriminator make predictions on the real images
            real_preds = ###Your code here

            #Calculate the discriminator loss using the real predictions and the real labels
            loss_D_real = ###Your code here, hint: you already defined the loss function and passed it as an argument to train_gan

            #Generate random noise for the generator
            z = ###Your code here, Hint: use torch.randn() and think about what shape this tensor should have to be fed into the generator.

            #Generate a fake image using your generator.
            fake_images = ###Your code here

            #Have the discriminator make predictions on the fake images
            fake_preds = ###Your code here, Hint: you should detach the fake images using .detach()

            #Calculate the discriminator loss using the fake predictions and the fake labels
            loss_D_fake = ###Your code here

            #Calculate the total discriminator loss
            loss_D = loss_D_real + loss_D_fake

            #Backpropagate the discriminator loss
            ###Your code here

            #Update the discriminator weights
            ###Your code here, Hint: use .step()

            #Zero out the gradients for the generator optimizer
            ###Your code here

            #Have the discriminator make predictions on the fake images again
            #This time you need gradients so do not .detach()
            fake_preds = ###Your code here

            #Calculate the generator loss on the fake predictions and the real labels using the loss function
            loss_G = ###Your code here

            #Backpropagate the generator loss
            ###Your code here

            #Update the generator weights
            ###Your code here

            #Print the losses for each batch
            if i % 100 == 0:
                clear_output(wait=True)
                print(f"Epoch [{epoch+1}/{n_epochs}] | Batch [{i+1}/{len(dataloader)}] | Loss_D: {loss_D:.4f} | Loss_G: {loss_G:.4f}")
                generate_images(generator, 10, z_dim, device)


Now that we have our model defined, we need some data to train our model. Let's use the MNIST dataset and create a dataloader that transforms the images to a tensor and normalizes them to -1 to 1 to match the expected range of outputs from the generator. (1 point)

In [9]:
#Create a transform using transforms.Compose to cast the data to a tensor and to normalize the data 
#Hint: the values in normalize should all be 0.5
transform = ###Your code here

#We will use the datasets.MNIST database, no need to write any code here
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

#Finally you can create your dataloader using your dataset from above
#Make sure shuffle is set to true
dataloader = ###Your code here


Now you should just be able to call `train_gan()` with all of the appropriate inputs. You should have defined all the arguments already. (1 point)

In [ ]:
###Your code here, call train_gan with the appropriate arguments

If you training was successful then you should be able to run your `generate_images` function to plot a range of samples from your model.

In [ ]:
generate_images(generator, 10, z_dim, device)

Now let's see what happens if we try to interpolate between random latent vectors! (1 point)

For this section you will need to first generate two random vectors `z1` and `z2`
>Hint: use torch.randn()

Then you need to create a set of interpolated latent vectors between `z1` and `z2`. You should create 10 steps between the latent codes to sample.
>Hint: use [torch.linspace()](https://pytorch.org/docs/stable/generated/torch.linspace.html) and [torch.lerp()](https://pytorch.org/docs/stable/generated/torch.lerp.html) to generate these latent vectors.

Then you need to use your trained generator to create new images given each of these latent vectors.

In [ ]:
#First you need to generate two random vectors with the shape (1, z_dim)
z1 = torch.randn(1, z_dim).to(device)
z2 = torch.randn(1, z_dim).to(device)


steps = 10
interpolation_range = torch.linspace(0, 1, steps=steps).unsqueeze(1).to(device)
print(interpolation_range.shape)
interpolated_z = torch.lerp(z1, z2, interpolation_range).squeeze(0).to(device)

#Now let's generate images for the interpolated latent vectors
interpolated_images = generator(interpolated_z)
print(interpolated_images.shape)


In [ ]:
#Now let's plot the interpolated images
fig, axes = plt.subplots(1, steps, figsize=(10, 10))
for i in range(steps):
    axes[i].imshow(interpolated_images[i].squeeze().detach().cpu(), cmap='gray')
    axes[i].axis('off')
plt.show()


#### Part 2 | Conditional GAN | pix2pix

In this section you will be building a **cGAN** based on the paper [Image-to-Image Translation with Conditional Adversarial Networks](https://arxiv.org/abs/1611.07004) by Isola et. al. The code here is adapted from [Philip Isola's github repo](https://github.com/phillipi/pix2pix). 

The general setup for this model will look very similar to the unconditional GAN model above. You will again have a `Generator` and a `Discriminator` model, however this time you will need to feed the input image into the discriminator model as your condition.

![pix2pix](imgs/pix2pix.png)

The `Generator` used is a `U-Net` architecture. A `U-Net` model architeecture is described in [U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/abs/1505.04597) and follows a symmetrical convolutional encoder-decoder architecture. It also includes skip connections to allow information to 'shortcut' across the network. 



The `Discriminator` uses a convolutional `PatchGAN` classifier. The `PatchGAN` is claimed by the authors to work on a wider range of problems since it only penalizes structure at the scale of image patches. Here an image patch is just meant to be some smaller subsection of an image. The best results from the paper were achieved using a PatchGAN with 'receptive-fields' that were 70 pixels x 70 pixels. 

![patch_gan.png](imgs/patch_gan.png)

In the paper they describe the network architecture using a compact notation:

>Let Ck denote a Convolution-BatchNorm-ReLU layer  with k filters. CDk denotes a Convolution-BatchNormDropout-ReLU layer with a dropout rate of 50%. All convolutions are 4 × 4 spatial filters applied with stride 2. Convolutions in the encoder, and in the discriminator, downsample  by a factor of 2, whereas in the decoder they upsample by a  factor of 2.

>The encoder-decoder architecture consists of:  encoder:  `C64-C128-C256-C512-C512-C512-C512-C512`  decoder:  `CD512-CD512-CD512-C512-C256-C128-C64`  After the last layer in the decoder, a convolution is applied to map to the number of output channels (3 in general,  except in colorization, where it is 2), followed by a Tanh  function. As an exception to the above notation, BatchNorm is not applied to the first C64 layer in the encoder.  All ReLUs in the encoder are leaky, with slope 0.2, while  ReLUs in the decoder are not leaky.  The U-Net architecture is identical except with skip connections between each layer i in the encoder and layer n − i  in the decoder, where n is the total number of layers. The  skip connections concatenate activations from layer i to  layer n − i. This changes the number of channels in the  decoder:  U-Net decoder:  `CD512-CD1024-CD1024-C1024-C1024-C512-C256-C128`

>The 70 × 70 discriminator architecture is:  `C64-C128-C256-C512`  After the last layer, a convolution is applied to map to  a 1-dimensional output, followed by a Sigmoid function.  As an exception to the above notation, BatchNorm is not  applied to the first C64 layer. All ReLUs are leaky, with  slope 0.2.

In these modules I have written out all of the steps in sequence to make it easier to follow the flow of data through the model. In practice you should write some functions that generate these models automatically using some other kind of logic (i.e. for loops and nn.Sequential()). The code from github does this model generation in a way that handles a variety of scenarios, however it is not the most straightforward code to read. The architecture defined below is designed for input images that are 256x256. If you try to run smaller images through the network you will get an error. 

In [5]:

class GeneratorUNet(nn.Module):
    def __init__(self, input_nc, output_nc):
        super(GeneratorUNet, self).__init__()
        self.e1 = nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1)

        self.e2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.be2 = nn.InstanceNorm2d(128)

        self.e3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.be3 = nn.InstanceNorm2d(256)

        self.e4 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1)
        self.be4 = nn.InstanceNorm2d(512)

        self.e5 = nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1)
        self.be5 = nn.InstanceNorm2d(512)

        self.e6 = nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1)
        self.be6 = nn.InstanceNorm2d(512)

        self.e7 = nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1)
        self.be7 = nn.InstanceNorm2d(512)

        self.e8 = nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1)
        self.be8 = nn.InstanceNorm2d(512)

        self.d1 = nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1)
        self.bd1 = nn.InstanceNorm2d(512)

        self.d2 = nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1)
        self.bd2 = nn.InstanceNorm2d(512)

        self.d3 = nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1)
        self.bd3 = nn.InstanceNorm2d(512)

        self.d4 = nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1)
        self.bd4 = nn.InstanceNorm2d(512)

        self.d5 = nn.ConvTranspose2d(1024, 256, kernel_size=4, stride=2, padding=1)
        self.bd5 = nn.InstanceNorm2d(256)

        self.d6 = nn.ConvTranspose2d(512, 128, kernel_size=4, stride=2, padding=1)
        self.bd6 = nn.InstanceNorm2d(128)

        self.d7 = nn.ConvTranspose2d(256, 64, kernel_size=4, stride=2, padding=1)
        self.bd7 = nn.InstanceNorm2d(64)

        self.d8 = nn.ConvTranspose2d(128, output_nc, kernel_size=4, stride=2, padding=1)

    def forward(self, x):
        x1 = F.leaky_relu(self.e1(x), 0.2)  # Apply first convolution and Leaky ReLU activation
        x2 = F.leaky_relu(self.be2(self.e2(x1)), 0.2)  # Apply second convolution, instance normalization, and Leaky ReLU
        x3 = F.leaky_relu(self.be3(self.e3(x2)), 0.2)  # Apply third convolution, instance normalization, and Leaky ReLU
        x4 = F.leaky_relu(self.be4(self.e4(x3)), 0.2)  # Apply fourth convolution, instance normalization, and Leaky ReLU
        x5 = F.leaky_relu(self.be5(self.e5(x4)), 0.2)  # Apply fifth convolution, instance normalization, and Leaky ReLU
        x6 = F.leaky_relu(self.be6(self.e6(x5)), 0.2)  # Apply sixth convolution, instance normalization, and Leaky ReLU
        x7 = F.leaky_relu(self.be7(self.e7(x6)), 0.2)  # Apply seventh convolution, instance normalization, and Leaky ReLU
        x8 = F.leaky_relu(self.e8(x7), 0.2)  # Apply eighth convolution and Leaky ReLU

        x = F.relu(self.bd1(self.d1(x8)))  # Upsample with first transposed convolution and apply ReLU
        x = F.dropout2d(x, 0.5)  # Apply dropout for regularization
        x = F.relu(self.bd2(self.d2(torch.cat([x, x7], 1))))  # Concatenate with skip connection and apply second upsample
        x = F.dropout2d(x, 0.5)  # Apply dropout for regularization
        x = F.relu(self.bd3(self.d3(torch.cat([x, x6], 1))))  # Concatenate with skip connection and apply third upsample
        x = F.relu(self.bd4(self.d4(torch.cat([x, x5], 1))))  # Concatenate with skip connection and apply fourth upsample
        x = F.relu(self.bd5(self.d5(torch.cat([x, x4], 1))))  # Concatenate with skip connection and apply fifth upsample
        x = F.relu(self.bd6(self.d6(torch.cat([x, x3], 1))))  # Concatenate with skip connection and apply sixth upsample
        x = F.relu(self.bd7(self.d7(torch.cat([x, x2], 1))))  # Concatenate with skip connection and apply seventh upsample
        x = F.tanh(self.d8(torch.cat([x, x1], 1)))  # Concatenate with skip connection and apply final transposed convolution with Tanh activation
        return x
    
class DiscriminatorPatchGAN(nn.Module):
    def __init__(self, input_nc, condition_nc):
        super(DiscriminatorPatchGAN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_nc + condition_nc, 64, kernel_size=4, stride=2, padding=1),  # Combine input and condition
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)  # Output a single value
        )

    def forward(self, img, condition):
        # Concatenate the input image and condition along the channel dimension
        x = torch.cat([img, condition], 1)
        return self.model(x)

Again, we will initialize our generator and discriminator models. 

In [6]:
generator_unet = GeneratorUNet(input_nc=3, output_nc=3).to(device)
discriminator_patchgan = DiscriminatorPatchGAN(input_nc=3, condition_nc=3).to(device)

In [ ]:
torchinfo.summary(generator_unet, (1, 3, 256, 256), device=device)

In [ ]:
torchinfo.summary(discriminator_patchgan,((1, 3, 256, 256), (1,3, 256, 256)), device=device)


Now we need to initialize the weights of the networks to match the paper. We will just use the init_weights function they defined. 

In [ ]:
def init_weights(net, init_type='normal', init_gain=0.02):
    """Initialize network weights.

    Parameters:
        net (network)   -- network to be initialized
        init_type (str) -- the name of an initialization method: normal | xavier | kaiming | orthogonal
        init_gain (float)    -- scaling factor for normal, xavier and orthogonal.

    We use 'normal' in the original pix2pix and CycleGAN paper. But xavier and kaiming might
    work better for some applications. Feel free to try yourself.
    """
    def init_func(m):  # define the initialization function
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            if init_type == 'normal':
                init.normal_(m.weight.data, 0.0, init_gain)
            elif init_type == 'xavier':
                init.xavier_normal_(m.weight.data, gain=init_gain)
            elif init_type == 'kaiming':
                init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
            elif init_type == 'orthogonal':
                init.orthogonal_(m.weight.data, gain=init_gain)
            else:
                raise NotImplementedError('initialization method [%s] is not implemented' % init_type)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)
        elif classname.find('BatchNorm2d') != -1:  # BatchNorm Layer's weight is not a matrix; only normal distribution applies.
            init.normal_(m.weight.data, 1.0, init_gain)
            init.constant_(m.bias.data, 0.0)

    print('initialize network with %s' % init_type)
    net.apply(init_func)  # apply the initialization function <init_func>



In [10]:
lr = 0.0002
beta1 = 0.5 #momentum
beta2 = 0.999 #alpha

init_weights(generator_unet, init_type='normal', init_gain=0.02)
init_weights(discriminator_patchgan, init_type='normal', init_gain=0.02)

g_optimizer = optim.Adam(generator_unet.parameters(), lr=lr, betas=(beta1, beta2))
d_optimizer = optim.Adam(discriminator_patchgan.parameters(), lr=lr, betas=(beta1, beta2))

Now let's download some data and create the dataloader. You will be using the facades dataset for this homework since it is nice and small.

In [ ]:

def download_facades(url, save_path):
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    if not os.path.exists(save_path):
        print(f"Downloading dataset from {url}...")
        urllib.request.urlretrieve(url, save_path)
        print("Download complete.")
    else:
        print("Dataset already downloaded.")

# URL for the edges2shoes dataset
facades_url = 'http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/facades.tar.gz'
save_path = './datasets/facades.tar.gz'

download_facades(facades_url, save_path)

In [ ]:
# Extract the dataset
import tarfile

def extract_dataset(tar_path, extract_path):
    with tarfile.open(tar_path, 'r:gz') as tar:
        tar.extractall(path=extract_path)

if not os.path.exists('./datasets/facades'):
    extract_dataset(save_path, './datasets')
else:
    print("Dataset already extracted.") 

Here, instead of using an existing dataset we will build our own. The first step to creating a dataset is defining a class that inherits from `Dataset` which is part of torch.utils.data. This class has a few critical sections. 

The first is __init__ which is called when a new FacadesDataset is generated. 

Inside of this particular dataset class there are a few internal variables declared. Root is the directory that contains the images for the dataset. Transform stores the transformations that should be applied each time __getitem__ is called. Images is a list that stores all of the paths to the image files of interest. 

Inside of __init__ is also a for loop that is run once when the FacadesDataset is created.

The next important section is __len__. This just allows you to query the dataset to figure out its length using len(dataset).

Finally the __getitem__ function is defined. This allows you to get a particular item from the dataset by index. In this FacadesDataset calling __getitem__ opens the image at the file location stored in self.images = [][idx] and applies any given transformations to the image. This is very useful if your images are large or you have a big batch size as it prevents you from having to load all of your images into memory on your computer.

The class is just a placeholder, it does not do anything until it is instantiated which we will do below.

In [13]:
class FacadesDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.images = []

        for root, dirs, files in os.walk(root):
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):
                    self.images.append(os.path.join(root, file))

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        if self.transform:
            image = self.transform(image)
        return image

Now that you have your Dataset class defined you can instantiate it. To create your dataset you need to pass in the folder location of your images and the transforms you wish to apply to the images. 

In [ ]:
#We will define the transforms to convert our image to a tensor and to normalize our image data.
transform = transforms.Compose([    
    transforms.ToTensor(), # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5),(0.5,0.5,0.5))
])

#Now we can actually create our dataset.
facades_dataset = FacadesDataset(root='./datasets/facades/', transform=transform)

#We can get the length of the dataset using len()
print(len(facades_dataset))

#We can get a particular item from the dataset using an idx
print(facades_dataset[0])

#The dataloader is another special function that pytorch uses for creating mini-batches of data for training.
#Batch size sets the number of samples that should be returned each time the dataloader is iterated. 
#Shuffle = True causes the idx or list of idx to be randomized for each batch, otherwise the samples are called in order
facades_dataloader = DataLoader(facades_dataset, batch_size=1, shuffle=True)

#After you have created your DataLoader 
#You can manually iterate through the newly defined dataloader using the syntax below. 
sample = next(iter(facades_dataloader))
print(sample.shape)

#The dataloader is what we will pass into our training function.


Let's look at a sample from the facades dataset.

In [ ]:
#Here we need to permute the axes of the image to order them correctly for plt.imshow()
#We also un-normalize the image so the color is not shifted when we draw the image
plt.imshow((sample[0].permute(1, 2, 0) + 1.0) / 2.0)
plt.show()

Here we see that the dataset is set up with pairs of images composed together into one frame. This means that we just need to choose half of our image to be fed into the generator and the other half will serve as the ground truth. 

Before we define the training loop we can also set up a few extra utility functions for plotting and doing some formatting.

In [16]:
#This is just a wrapper function for the loss function
def GAN_loss(output, target):
    criterion = nn.BCEWithLogitsLoss()
    loss = criterion(output, target)
    return loss

#This is another wrapper function
def l1_loss(output, target):
    criterion = nn.L1Loss()
    loss = criterion(output, target)
    return loss

#This function will be used to plot a set of images.
def plot_sample_grid(condition, fake_photo, real_photo):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    axs[0].imshow(condition)
    axs[0].set_title('Condition')
    axs[1].imshow(fake_photo)
    axs[1].set_title('Fake Photo')
    axs[2].imshow(real_photo)
    axs[2].set_title('Real Photo')
    plt.show()

#This function will help when we want to turn on or off the gradient tracking for our parameters
def set_requires_grad(model, requires_grad=True):
    for param in model.parameters():
        param.requires_grad = requires_grad

#This function simplifies converting from a tensor to an image and does the un-normalization so we can see the correct colors
#From https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/util/util.py
def tensor2im(input_image, imtype=np.uint8):
    """"Converts a Tensor array into a numpy image array.

    Parameters:
        input_image (tensor) --  the input image tensor array
        imtype (type)        --  the desired type of the converted numpy array
    """
    if not isinstance(input_image, np.ndarray):
        if isinstance(input_image, torch.Tensor):  # get the data from a variable
            image_tensor = input_image.data
        else:
            return input_image
        image_numpy = image_tensor.cpu().float().numpy()  # convert it into a numpy array
        if image_numpy.shape[0] == 1:  # grayscale to RGB
            image_numpy = np.tile(image_numpy, (3, 1, 1))
        image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0  # post-processing: tranpose and scaling
    else:  # if it is a numpy array, do nothing
        image_numpy = input_image
    return image_numpy.astype(imtype)

Now we can move on to defining the training loop.

In this model you will be trying to go from facade labels to photos of facades. The goal of the generator is to create realistic looking images of facades, given a labeled facade image. The goal of the discriminator is to determine if the generated image is a real photo of a facade or a generated photo of a facade. If you would like to swap the order to generate labels from images you can do so by modifying how the data is sliced.

In [17]:
def train_pix2pix(generator, discriminator, dataloader, device, lambda_l1=100, num_epochs=25):
    for epoch in tqdm(range(num_epochs)):
        for i, data in enumerate(dataloader):
            #Take half of the image as the input and the other half as the ground truth
            real_photo = data[..., :256].to(device)
            condition = data[..., 256:].to(device) 

            # Start Generation Here
            # Train Discriminator
            set_requires_grad(discriminator, True)
            d_optimizer.zero_grad()

            # Fake pairs
            fake_photo = generator(condition)
            pred_fake = discriminator(fake_photo.detach(), condition)
            fake = torch.zeros_like(pred_fake, device=device)
            loss_D_fake = GAN_loss(pred_fake, fake)

            # Real pairs
            pred_real = discriminator(real_photo, condition) #The discriminator takes in the real/fake photo and the condition
            valid = torch.ones_like(pred_real, device=device)
            loss_D_real = GAN_loss(pred_real, valid)    

            # Total Discriminator loss
            loss_D = (loss_D_real + loss_D_fake) / 2
            loss_D.backward()
            d_optimizer.step()
            
            # Train Generator
            set_requires_grad(discriminator, False)
            g_optimizer.zero_grad()
            pred_fake = discriminator(fake_photo, condition)
            loss_G_GAN = GAN_loss(pred_fake, valid)
            loss_G_L1 = l1_loss(fake_photo, real_photo) * lambda_l1
            loss_G = loss_G_GAN + loss_G_L1
            loss_G.backward()
            g_optimizer.step()

            
            # Start Generation Here        
            if i % 50 ==0:
                clear_output(wait=True)
                print(f"Epoch {epoch+1}/{num_epochs}, Loss_D: {loss_D.item()}, Loss_G_GAN: {loss_G_GAN.item()}, Loss_G_L1: {loss_G_L1.item()}")
                with torch.no_grad():            
                    fake_image = generator(condition)[0]
                    fake_photo_img = tensor2im(fake_image)
                    real_photo_img = tensor2im(real_photo[0])
                    plot_sample_grid(tensor2im(condition.squeeze(0)), fake_photo_img, real_photo_img)  


Finally we can call our train_pix2pix function and proceed with training our model. 

In [ ]:
train_pix2pix(generator_unet, discriminator_patchgan, facades_dataloader, device)


Great! With any luck (and a bit of training time) you should begin to see improvements in the generation of fake photos given a labeled condition image as input. 

### Part 3 | pix2pix with your own dataset | 5 points

In this section of the assignment you will need to create your own dataset and use it for training a pix2pix model. Your dataset needs to have a minimum of 30 pairs of images. With only 30 image pairs you likely won't get amazing results from your model, but this will help serve as a template if you would like to build a larger dataset at a later time. 

I will leave it up to you to decide how you would like to generate your image pairs, but the size of your images should be 256x256 pixels and the image pairs should be split into two folders where the order of images in one folder matches the order in the other folder. I.e. you should have a folder of images_A and a folder of images_B and the filenames in each folder should be set up such that they are in the same order in both folders. Using a numerical file naming scheme is your best approach here. If you choose to have one or both of your datasets be in grayscale then you will need to modify the input/output number of channels. 

You should add your image folders to the same folder that contains the previously downloaded datasets. 

The rest of the setup of the pix2pix model has already been done for you above so the main thing you need to do is to define the dataset.

In [ ]:
#Here I have already defined the DIY dataset, note the differences with the facade dataset class
class DIY_Dataset(Dataset):
    def __init__(self, root_A, root_B, transform=None):
        self.root_a = root_A
        self.root_b = root_B
        self.transform = transform
        self.images_A = []
        self.images_B = []

        for root_A, dirs, files in os.walk(root_A):
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):
                    self.images_A.append(os.path.join(root_A, file))
        
        for root_B, dirs, files in os.walk(root_B):
            for file in files:
                if file.endswith('.jpg') or file.endswith('.png'):
                    self.images_B.append(os.path.join(root_B, file))

    def __len__(self):
        return min(len(self.images_A),len(self.images_B))
    
    def __getitem__(self, idx):
        image_a = Image.open(self.images_A[idx])
        image_b = Image.open(self.images_B[idx])
        if self.transform:
            image_a = self.transform(image_a)
            image_b = self.transform(image_b)
        return image_a, image_b     

In [ ]:
#We will define the transforms to convert our image to a tensor and to normalize our image data.
transform_DIY = transforms.Compose([    
    transforms.ToTensor(), # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5),(0.5,0.5,0.5))
])

#Now we can actually create our dataset.
DIY_dataset = ###Your code here

#We can get the length of the dataset using len()
print(len(DIY_dataset))

#We can get a particular item from the dataset using an idx
print(DIY_dataset[0][0].shape,DIY_dataset[1][0].shape)

#Define you dataloader
DIY_dataloader = DataLoader(DIY_dataset, batch_size=1, shuffle=True)

In [ ]:
def train_pix2pix_DIY(generator, discriminator, dataloader, d_opt, g_opt, device, lambda_l1=100, num_epochs=25):
    for epoch in tqdm(range(num_epochs)):
        for i, data in enumerate(dataloader):
            #Your dataloader should be set up to return two images instead of one combined image that needs to be split in two.
            #If you would like to switch the order of your model then you can just swap these indices
            real_photo = data[0].to(device)
            condition = data[1].to(device) 

            # Start Generation Here
            # Train Discriminator
            set_requires_grad(discriminator, True)
            d_opt.zero_grad()

            # Fake pairs
            fake_photo = generator(condition)
            pred_fake = discriminator(fake_photo.detach(), condition)
            fake = torch.zeros_like(pred_fake, device=device)
            loss_D_fake = GAN_loss(pred_fake, fake)

            # Real pairs
            pred_real = discriminator(real_photo, condition) #The discriminator takes in the real/fake photo and the condition
            valid = torch.ones_like(pred_real, device=device)
            loss_D_real = GAN_loss(pred_real, valid)    

            # Total Discriminator loss
            loss_D = (loss_D_real + loss_D_fake) / 2
            loss_D.backward()
            d_opt.step()
            
            # Train Generator
            set_requires_grad(discriminator, False)
            g_opt.zero_grad()
            pred_fake = discriminator(fake_photo, condition)
            loss_G_GAN = GAN_loss(pred_fake, valid)
            loss_G_L1 = l1_loss(fake_photo, real_photo) * lambda_l1
            loss_G = loss_G_GAN + loss_G_L1
            loss_G.backward()
            g_opt.step()
            
            # Start Generation Here        
            if i % 50 ==0:
                clear_output(wait=True)
                print(f"Epoch {epoch+1}/{num_epochs}, Loss_D: {loss_D.item()}, Loss_G_GAN: {loss_G_GAN.item()}, Loss_G_L1: {loss_G_L1.item()}")
                with torch.no_grad():            
                    fake_image = generator(condition)[0]
                    fake_photo_img = tensor2im(fake_image)
                    real_photo_img = tensor2im(real_photo[0])
                    plot_sample_grid(tensor2im(condition.squeeze(0)), fake_photo_img, real_photo_img)

In [ ]:
generator_unet_DIY = GeneratorUNet(input_nc=3, output_nc=3).to(device)
discriminator_patchgan_DIY = DiscriminatorPatchGAN(input_nc=3, condition_nc=3).to(device)

In [ ]:
lr = 0.0002
beta1 = 0.5 #momentum
beta2 = 0.999 #alpha

init_weights(generator_unet_DIY, init_type='normal', init_gain=0.02)
init_weights(discriminator_patchgan_DIY, init_type='normal', init_gain=0.02)

g_optimizer_DIY = optim.Adam(generator_unet_DIY.parameters(), lr=lr, betas=(beta1, beta2))
d_optimizer_DIY = optim.Adam(discriminator_patchgan_DIY.parameters(), lr=lr, betas=(beta1, beta2))

In [ ]:
train_pix2pix_DIY(generator_unet_DIY, discriminator_patchgan_DIY, g_optimizer_DIY, d_optimizer_DIY, DIY_dataloader, device)

Congratulations on completing HW3! To submit your assignment please save all your files and then commit and sync them to your github repo. Also create a zip file containing everything in the HW3 folder (make sure to include your image files!) and upload this zip file to canvas.